May 4, 2021

### Example of Baseline vs IB-GAN using the UCR dataset SpokenArabicDigits

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [ ]:
import keras
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [ ]:
import random
from numpy import zeros, ones, expand_dims
from numpy.random import randn, randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Embedding, Dense, LSTM, Multiply, Add, Lambda
from keras.layers import Reshape, Flatten, Activation, Concatenate
from keras.layers import Conv1D, ZeroPadding1D, MaxPooling1D, Conv2D, Conv2DTranspose, TimeDistributed
from keras.layers import BatchNormalization, Dropout, LeakyReLU, RepeatVector, ReLU, GlobalAveragePooling1D
from keras.initializers import RandomNormal
from matplotlib import pyplot
from keras.metrics import Precision, Recall, AUC
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import precision_recall_curve, average_precision_score, auc, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, balanced_accuracy_score, accuracy_score, f1_score
from itertools import chain


### Load & Process Data

In [ ]:
#State UCR dataset
filefolder = "SpokenArabicDigits"
trainx_key = 'UCR/' + filefolder + '/X_train.npy'
trainy_key ='UCR/' + filefolder + '/y_train.npy'
testx_key ='UCR/' + filefolder + '/X_test.npy'
testy_key ='UCR/' + filefolder + '/y_test.npy'

x_train = np.load(trainx_key)
y_train = np.load(trainy_key)
x_test = np.load(testx_key)
y_test = np.load(testy_key)


(6599, 93, 13) (6599, 1) (2199, 93, 13) (2199, 1)


In [ ]:
#See class frequencies 
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([660, 660, 660, 660, 660, 660, 660, 660, 660, 659]))

In [ ]:
#Introduce class imbalance
np.random.seed(2021)
minority_index = [k for k in range(len(y_train)) if y_train[k] in [0, 1, 2, 3, 4]]
toRemove = [i for i in minority_index if np.random.rand(1) < 0.75]
x_train = np.delete(x_train, toRemove, axis=0)
y_train = np.delete(y_train, toRemove, axis=0)

minority_index = [k for k in range(len(y_test)) if y_test[k] in [0, 1, 2, 3, 4]]
toRemove = [i for i in minority_index if np.random.rand(1) < 0.75]
x_test = np.delete(x_test, toRemove, axis=0)
y_test = np.delete(y_test, toRemove, axis=0)

In [ ]:
#See class frequencies after imbalance
uniq_vals, uniq_counts = np.unique(y_train, return_counts=True)
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([158, 175, 162, 176, 176, 660, 660, 660, 660, 659]))

In [ ]:
#Set class weights
weights = np.zeros(len(uniq_vals))
class_weight = {}
for i in list(range(len(uniq_vals))):
    weights[i] = y_train.shape[0]/(len(uniq_vals) * uniq_counts[i])
    class_weight.update({uniq_vals[i] : weights[i]})
print(class_weight)

{0: 2.6240506329113926, 1: 2.369142857142857, 2: 2.5592592592592593, 3: 2.355681818181818, 4: 2.355681818181818, 5: 0.6281818181818182, 6: 0.6281818181818182, 7: 0.6281818181818182, 8: 0.6281818181818182, 9: 0.6291350531107739}


In [ ]:
#Training parameters
n_classes = len(uniq_vals)
n_epochs= 10
n_samples = max(int(x_train.shape[0]/5), 2*n_classes)
p_missing = 0.1


#Set parameters of data
max_sequence_length = x_train.shape[1]
n_features = x_train.shape[2]
k = 2


In [ ]:
#Create onehot
y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=n_classes)

### 1. Baseline Classifier

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def single_CNN(ts_shape = (max_sequence_length, n_features)):
    # time series input
    in_ts = Input(shape=ts_shape)
    filter1 = n_features * 2
    fmaps = n_classes*2
    fe = Conv1D(filter1, kernel_size=k, padding='same', input_shape = (max_sequence_length, n_features))(in_ts)
    fe = BatchNormalization()(fe)
    fe = MaxPooling1D(pool_size=(2))(fe)
    fe = Conv1D(1, kernel_size=k, padding='same', input_shape = (max_sequence_length, n_features))(fe)
    fe = BatchNormalization()(fe)
    fe = MaxPooling1D(pool_size=(2))(fe)
    #Fully connected layers
    fe = Flatten()(fe)
    fe = Dense(fmaps)(fe)
    out_labels = Dense(n_classes, activation='softmax', name="cnn_labels")(fe)

    model = Model(inputs=in_ts, outputs=out_labels, name="CNN_Classifier")
    opt = Adam()
    losses = {"cnn_labels":'categorical_crossentropy'}
    model.compile(loss=losses, optimizer=opt, metrics = ['categorical_accuracy']) 
    model.summary() #prints out layers of model
    return model

baseline_classifier = single_CNN()

Model: "CNN_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 93, 13)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 93, 26)            702       
_________________________________________________________________
batch_normalization_1 (Batch (None, 93, 26)            104       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 26)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 1)             53        
_________________________________________________________________
batch_normalization_2 (Batch (None, 46, 1)             4         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 23, 1)          

In [ ]:
#Run one iteration of baseline classifier
tf.keras.backend.clear_session()
random.seed(123)
baseline_classifier = single_CNN()
baseline_classifier.fit(x_train, y_train_onehot,verbose=1)
pred_cnn = baseline_classifier.predict(x_test)
rounded_cnn_labels= np.argmax(pred_cnn, axis=1)

baseline_metrics = pd.DataFrame({'ClassifierType': "Baseline No Class Weights",
                            'Accuracy': accuracy_score(y_test, rounded_cnn_labels), 
                             'Precision': precision_score(y_test, rounded_cnn_labels, average="macro"),
                             'Recall': recall_score(y_test, rounded_cnn_labels, average="macro"),
                             'BalancedAccuracy': balanced_accuracy_score(y_true=y_test, y_pred = rounded_cnn_labels),
                             'F1-score': f1_score(y_true=y_test, y_pred=rounded_cnn_labels, average="macro")
                            }, index=[0])
print(baseline_metrics)

Model: "CNN_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 93, 13)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 93, 26)            702       
_________________________________________________________________
batch_normalization_1 (Batch (None, 93, 26)            104       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 26)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 1)             53        
_________________________________________________________________
batch_normalization_2 (Batch (None, 46, 1)             4         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 23, 1)          

In [ ]:
#Run one iteration of baseline classifier with class weights
tf.keras.backend.clear_session()
random.seed(123)
baseline_classifier = single_CNN()
baseline_classifier.fit(x_train, y_train_onehot,class_weight = class_weight, verbose=1)
pred_cnn = baseline_classifier.predict(x_test)
rounded_cnn_labels= np.argmax(pred_cnn, axis=1)

baseline_metrics = pd.DataFrame({'ClassifierType': "Baseline with Class Weights",
                            'Accuracy': accuracy_score(y_test, rounded_cnn_labels), 
                             'Precision': precision_score(y_test, rounded_cnn_labels, average="macro"),
                             'Recall': recall_score(y_test, rounded_cnn_labels, average="macro"),
                             'BalancedAccuracy': balanced_accuracy_score(y_true=y_test, y_pred = rounded_cnn_labels),
                             'F1-score': f1_score(y_true=y_test, y_pred=rounded_cnn_labels, average="macro")
                            }, index=[0])
print(baseline_metrics)

Model: "CNN_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 93, 13)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 93, 26)            702       
_________________________________________________________________
batch_normalization_1 (Batch (None, 93, 26)            104       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 26)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 1)             53        
_________________________________________________________________
batch_normalization_2 (Batch (None, 46, 1)             4         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 23, 1)          

### 2. Imputation Balanced GAN

In [ ]:
def define_classifier(ts_shape=(max_sequence_length, n_features),
                           hint_shape = (max_sequence_length, n_features)):
    # time series input
    in_ts = Input(shape=ts_shape)
    in_hint = Input(shape= hint_shape)
    filter1 = n_features * 2
    fmaps = n_classes*2
    fe = Conv1D(filter1, kernel_size=k, padding='same', input_shape = (max_sequence_length, n_features))(in_ts)
    fe = BatchNormalization()(fe)
    fe = MaxPooling1D(pool_size=(2))(fe)
    fe = Conv1D(1, kernel_size=k, padding='same', input_shape = (max_sequence_length, n_features))(fe)
    fe = BatchNormalization()(fe)
    fe = MaxPooling1D(pool_size=(2))(fe)
    #Fully connected layers
    fe = Flatten()(fe)
    fe = Dense(fmaps)(fe)
    out_labels = Dense(n_classes, activation='softmax', name="labels_output")(fe)
    model = Model(inputs=[in_ts, in_hint], outputs=out_labels, name="Labels_Classifier")
    opt = Adam()
    losses = {"labels_output":'categorical_crossentropy'}
    model.compile(loss=losses, optimizer=opt, metrics = ['categorical_accuracy']) 
    model.summary() #prints out layers of model
    return model


In [ ]:
def define_discriminator(ts_shape=(max_sequence_length, n_features), 
                           hint_shape = (max_sequence_length, n_features)):
    # time series input
    in_ts = Input(shape=ts_shape)
    in_hint = Input(shape= hint_shape)
    in_merged = Concatenate(axis=2)([in_ts, in_hint])
    filter1 = n_features * 2
    fe = Conv1D(filter1, kernel_size=k, padding="same", activation='relu', 
                input_shape = (max_sequence_length, n_features))(in_merged)
    fe = Conv1D(n_features, kernel_size=k, padding="same",
                input_shape = (max_sequence_length, n_features))(fe)
    # real/fake output
    out_realfake = Activation('sigmoid', name="realfake_output")(fe)
    model = Model(inputs=[in_ts, in_hint], outputs=out_realfake, name="RealFake_Discriminator")
    # compile model
    opt = Adam()
    losses = {"realfake_output":'binary_crossentropy'}
    model.compile(loss=losses, optimizer=opt, metrics = ['binary_accuracy']) 
    model.summary() #prints out layers of model
    return model

In [ ]:
def define_generator_impute(missing_shape = (max_sequence_length, n_features),
                            mask_shape = (max_sequence_length, n_features),
                            labels_shape = (n_classes, ),
                            hint_shape = (max_sequence_length, n_features)):
    # time series input
    in_ts = Input(shape=missing_shape)
    in_hint = Input(shape= hint_shape)

    #Other inputs
    in_labels = Input(shape = labels_shape)
    labels_tile = RepeatVector(max_sequence_length)(in_labels)
    in_mask = Input(shape=mask_shape)
    in_merged = Concatenate(axis=2)([in_ts, in_hint, labels_tile])
    
    filter1 = n_features * 2
    fe = Conv1D(filter1, kernel_size=k, padding="same", activation='relu', 
                input_shape = (max_sequence_length, n_features))(in_merged)
    fe = Conv1D(n_features, kernel_size=k, padding="same",
                input_shape = (max_sequence_length, n_features))(fe)
    fake_ts = fe 
    Masked_Data = Multiply()([in_ts, in_mask])
    Reversed_Mask = Lambda(lambda x: 1. - x)(in_mask)
    Imputed_Vals = Multiply()([fake_ts, Reversed_Mask])
    imputed_ts = Add(name = "fakets_output")([Masked_Data, Imputed_Vals]) #Note that we replaced noise with values from our imputed copy
    # define model
    model = Model(inputs = [in_ts, in_hint, in_mask, in_labels], outputs = [imputed_ts, in_hint], name="Generator") 
    model.summary()
    return model
    

In [ ]:
def define_gan(g_model, d_model, c_model):
    # make weights in the discriminator AND classifier not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False  
    discriminator_output = d_model(g_model.output)
    classifier_output = c_model(g_model.output)
    model = Model(inputs = g_model.input, output = [classifier_output, discriminator_output])
    # compile model
    opt = Adam()
    twolosses = {"Labels_Classifier":'binary_crossentropy', "RealFake_Discriminator":'binary_crossentropy'}
    twolossesweights = {"Labels_Classifier": 1, "RealFake_Discriminator": 1}
    model.compile(loss=twolosses, optimizer=opt, loss_weights = twolossesweights)
    model.summary()    
    return model


In [ ]:
def sample_toImpute(x_train, y_train, p_missing, n_samples=n_samples, 
                             n_features = n_features, max_sequence_length=max_sequence_length):
    #1. Reweighted Sampling
    uniq_classes, uniq_counts = np.unique(y_train, return_counts=True)
    class_probs = uniq_counts/len(y_train)
    inv_probs = (1/class_probs)/(sum(1/class_probs))
    sample_counts = np.round(inv_probs * n_samples)
    sample_counts[len(sample_counts) - 1] = n_samples - sum(sample_counts[0:(len(sample_counts) - 1)])                           
    ix_bal = []
    for i in uniq_classes:
        index_class = [k for k in range(len(y_train)) if y_train[k] == i]
        index_sample = random.choices(index_class, k=int(sample_counts[int(i)]))
        ix_bal.append(index_sample)
    
    ix_bal = list(chain(*ix_bal))
    
    impute_y_label = y_train[ix_bal]
    impute_y_label = tf.keras.utils.to_categorical(impute_y_label, num_classes=n_classes)
    
    #2. Random Masking under MCAR scheme
    #Create mask matrix: M_{ij} = 1 mean data is real
    impute_mask = np.random.rand(max_sequence_length * n_features * n_samples) > p_missing
    impute_mask = 1*impute_mask.reshape(n_samples, max_sequence_length, n_features)
    
    #Mask the true data with noise values 
    impute_noise = np.random.rand(max_sequence_length * n_features * n_samples)
    impute_noise = impute_noise.reshape(n_samples, max_sequence_length, n_features)
    impute_x_train = x_train[ix_bal]
    impute_z_input = impute_x_train * (impute_mask) + impute_noise * (1-impute_mask)
    
    return impute_z_input, impute_mask, impute_y_label, impute_x_train

In [ ]:
def sample_hint_matrix(mask_matrix, p_hint=0.8):
    hints = np.random.rand(mask_matrix.shape[0] * mask_matrix.shape[1] * mask_matrix.shape[2]) > p_missing
    hints = 1*hints.reshape(mask_matrix.shape[0], mask_matrix.shape[1], mask_matrix.shape[2])
    hint_matrix = hints*mask_matrix
    return(hint_matrix)

In [ ]:
def shuffle_minibatch(X_both, labels_both, hint_both):
    p = np.random.permutation(X_both.shape[0])
    X_both = X_both[p]
    labels_both = labels_both[p]
    hint_both = hint_both[p]
    return X_both, labels_both, hint_both

In [ ]:
%time
def train(g_model, d_model, c_model, gan_model, 
          x_train, y_train, x_test, n_epochs, p_missing,
          n_features = n_features, max_sequence_length = max_sequence_length, n_samples=n_samples):

    n_steps = int((x_train.shape[0]/n_samples) * n_epochs)
    n_burn = 10
    # manually enumerate epochs
    print("Number of steps total: ", n_steps)
    for i in range(n_steps):  
        
        # Select a random batch of true data
        idx = np.random.randint(0, x_train.shape[0], n_samples)
        true_samples = x_train[idx]

        true_labels = y_train[idx]
        true_labels = tf.keras.utils.to_categorical(true_labels, num_classes=n_classes)
        
        #Select generator input:
        z_input, z_mask, z_labels, z_train = sample_toImpute(x_train=x_train, y_train=y_train,
                                                             p_missing = p_missing,n_samples=n_samples, 
                                                             n_features = n_features,
                                                             max_sequence_length=max_sequence_length)
        #Discriminator ground truths
        true_M = np.ones((n_samples, max_sequence_length, n_features))
        fake_M = z_mask
        true_hint = sample_hint_matrix(true_M, p_hint=0.5)
        z_hint = sample_hint_matrix(z_mask, p_hint=0.5)

        #Get generator prediction
        gen_samples, gen_hint = generator.predict([z_input, z_hint, z_mask, z_labels])
        
        # Update discriminator
        d_loss_real = discriminator.train_on_batch([true_samples, true_hint], true_M)
        d_loss_fake = discriminator.train_on_batch([gen_samples, gen_hint], fake_M)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        
        # Train the classifier after burning the first n iterations
        if i <= n_burn:
            c_loss = classifier.train_on_batch([true_samples, true_hint], true_labels)
        if i > n_burn:
            samples_both = np.concatenate((true_samples, gen_samples), axis=0)
            labels_both = np.concatenate((true_labels, z_labels), axis=0)
            hint_both = np.concatenate((true_hint, z_hint), axis=0)
            samples_both, labels_both, hint_both = shuffle_minibatch(samples_both, labels_both, hint_both)
            c_loss = classifier.train_on_batch([samples_both, hint_both], labels_both)

        # Train the generator
        g_loss = gan_model.train_on_batch([z_input, z_hint, z_mask, z_labels], [z_labels, true_M])
        
        # State losses
        print ("%d [D loss: %f] [G loss: %f] [C loss: %f]" % (i, d_loss[0], g_loss[0], c_loss[0]))

        if i  == (n_steps -1):
            test_M = np.ones((x_test.shape[0], max_sequence_length, n_features))
            test_hint = sample_hint_matrix(test_M, p_hint = 0.5)
            pred_realfake = d_model.predict([x_test, test_hint])
            pred_labels = c_model.predict([x_test, test_hint])
    return pred_realfake, pred_labels

CPU times: user 19 µs, sys: 24 µs, total: 43 µs
Wall time: 75.6 µs


#### Train IB-GAN for one iteration

In [ ]:
%%time
tf.keras.backend.clear_session()
random.seed(123)
#Hyperparameters
n_epochs = 100
p_missing = 0.1

#Loop
classifier = define_classifier()
discriminator = define_discriminator()
generator = define_generator_impute()
gan_model = define_gan(g_model = generator, d_model = discriminator, c_model = classifier)
pred_realfake, pred_labels = train(g_model = generator, d_model = discriminator, c_model = classifier, 
                               gan_model=gan_model, p_missing=p_missing,
                               x_train = x_train, y_train = y_train,
                               x_test = x_test, n_epochs=n_epochs)

rounded_labels= np.argmax(pred_labels, axis = 1)

pred_metrics = pd.DataFrame({'ClassifierType': "IB-GAN",
                    'Accuracy': accuracy_score(y_test, rounded_labels), 
                     'Precision': precision_score(y_test, rounded_labels, average="macro"),
                     'Recall': recall_score(y_test, rounded_labels, average="macro"),
                     'BalancedAccuracy': balanced_accuracy_score(y_true=y_test, y_pred = rounded_labels),
                     'F1-score': f1_score(y_true=y_test, y_pred=rounded_labels, average="macro")
                    }, index = [0])


Model: "Labels_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 93, 13)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 93, 26)            702       
_________________________________________________________________
batch_normalization_1 (Batch (None, 93, 26)            104       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 26)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 1)             53        
_________________________________________________________________
batch_normalization_2 (Batch (None, 46, 1)             4         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 23, 1)       

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 93, 13)       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 93, 13)       0                                            
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 93, 10)       0           input_7[0][0]                    
____________________________________________________________________________________________

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.749327] [G loss: 1.129638] [C loss: 2.777773]


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.720249] [G loss: 1.089097] [C loss: 2.769699]
2 [D loss: 0.690112] [G loss: 1.038612] [C loss: 2.671327]
3 [D loss: 0.660841] [G loss: 1.003426] [C loss: 2.617783]
4 [D loss: 0.631245] [G loss: 0.969167] [C loss: 2.556081]
5 [D loss: 0.602717] [G loss: 0.927545] [C loss: 2.584717]
6 [D loss: 0.575001] [G loss: 0.885251] [C loss: 2.528491]
7 [D loss: 0.547903] [G loss: 0.849937] [C loss: 2.411498]
8 [D loss: 0.522560] [G loss: 0.817405] [C loss: 2.410905]
9 [D loss: 0.496404] [G loss: 0.779333] [C loss: 2.355428]
10 [D loss: 0.472498] [G loss: 0.756427] [C loss: 2.339181]
11 [D loss: 0.448339] [G loss: 0.717516] [C loss: 2.331836]
12 [D loss: 0.424922] [G loss: 0.679316] [C loss: 2.241491]
13 [D loss: 0.402016] [G loss: 0.654098] [C loss: 2.241865]
14 [D loss: 0.381279] [G loss: 0.616325] [C loss: 2.218970]
15 [D loss: 0.360904] [G loss: 0.576536] [C loss: 2.143676]
16 [D loss: 0.341720] [G loss: 0.557010] [C loss: 2.134474]
17 [D loss: 0.324679] [G loss: 0.529212] [C loss:

In [ ]:
pred_metrics

,ClassifierType,Accuracy,Precision,Recall,BalancedAccuracy,F1-score
0,IB-GAN,0.790462,0.693231,0.672434,0.672434,0.65461
